# Patient Data Preprocessing
---

Reading and preprocessing patient data of the eICU dataset from MIT with the data from over 139k patients collected in the US.

This notebook addresses the preprocessing of the following eICU tables:
* patient
* note

## Importing the necessary packages

In [ ]:
import os                                  # os handles directory/workspace changes
import numpy as np                         # NumPy to handle numeric and NaN operations
import yaml                                # Save and load YAML files

In [ ]:
# Debugging packages
import pixiedust                           # Debugging in Jupyter Notebook cells

In [ ]:
# Change to parent directory (presumably "Documents")
os.chdir("../../../..")
# Path to the CSV dataset files
data_path = 'data/eICU/uncompressed/'
# Path to the code files
project_path = 'code/eICU-mortality-prediction/'

In [ ]:
# Make sure that every large operation can be handled, by using the disk as an overflow for the memory
!export MODIN_OUT_OF_CORE=true
# Another trick to do with Pandas so as to be able to allocate bigger objects to memory
!sudo bash -c 'echo 1 > /proc/sys/vm/overcommit_memory'

In [ ]:
import modin.pandas as pd                  # Optimized distributed version of Pandas
# import pandas as pd
import data_utils as du                    # Data science and machine learning relevant methods

Allow pandas to show more columns:

In [ ]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

Set the random seed for reproducibility

In [ ]:
du.set_random_seed(42)

## Patient data

### Initialize variables

In [ ]:
# List of categorical features
cat_feat = []
# Dictionary of the one hot encoded columns originary from each categorical feature, that will be embedded
cat_feat_ohe = dict()

### Read the data

In [ ]:
patient_df = pd.read_csv(f'{data_path}original/patient.csv')
patient_df.head()

In [ ]:
patient_df.dtypes

Convert dataframe to Pandas, as the `one_hot_encoding_dataframe` isn't working properly with Modin:

In [ ]:
patient_df, pd = du.utils.convert_dataframe(patient_df, to='pandas', dtypes=dict(patient_df.dtypes))

In [ ]:
patient_df.dtypes

In [ ]:
len(patient_df)

In [ ]:
patient_df.patientunitstayid.nunique()

In [ ]:
patient_df.patientunitstayid.value_counts()

Get an overview of the dataframe through the `describe` method:

In [ ]:
patient_df.describe().transpose()

In [ ]:
patient_df.columns

In [ ]:
patient_df.dtypes

### Check for missing values

In [ ]:
du.search_explore.dataframe_missing_values(patient_df)

### Remove unneeded features

Besides removing unneeded hospital and time information, I'm also removing the admission diagnosis (`apacheadmissiondx`) as it doesn't follow the same structure as the remaining diagnosis data (which is categorized in increasingly specific categories, separated by "|").

In [ ]:
patient_df = patient_df[['patientunitstayid', 'gender', 'age', 'ethnicity',  'admissionheight',
                         'hospitaldischargeoffset', 'hospitaldischargelocation', 'hospitaldischargestatus',
                         'admissionweight']]
patient_df.head()

### Make the age feature numeric

In the eICU dataset, ages above 89 years old are not specified. Instead, we just receive the indication "> 89". In order to be able to work with the age feature numerically, we'll just replace the "> 89" values with "90", as if the patient is 90 years old. It might not always be the case, but it shouldn't be very different and it probably doesn't affect too much the model's logic.

In [ ]:
patient_df.age.value_counts().head()

In [ ]:
# Replace the "> 89" years old indication with 90 years
patient_df.age = patient_df.age.replace(to_replace='> 89', value=90)

In [ ]:
patient_df.age.value_counts().head()

In [ ]:
# Make the age feature numeric
patient_df.age = patient_df.age.astype(float)

### Discretize categorical features

Convert binary categorical features into one hot encode columns, which can later be embedded or used as is.

#### One hot encode features

Update list of categorical features:

In [ ]:
patient_df.gender.value_counts()

In [ ]:
patient_df.gender = patient_df.gender.map(lambda x: 1 if x == 'Male' else 0 if x == 'Female' else np.nan)

In [ ]:
patient_df.gender.value_counts()

#### Separate and prepare features for embedding

Identify categorical features that have more than 5 unique categories, which will go through an embedding layer afterwards, and enumerate them.


Update list of categorical features and add those that will need embedding (features with more than 5 unique values):

In [ ]:
cat_feat = ['ethnicity']

In [ ]:
patient_df[cat_feat].head()

Apply one hot encoding:

In [ ]:
patient_df, new_columns = du.data_processing.one_hot_encoding_dataframe(patient_df, columns=cat_feat,
                                                                        join_rows=False,
                                                                        get_new_column_names=True,
                                                                        inplace=True)
patient_df

In [ ]:
patient_df.dtypes

Save the association between the original categorical features and the new one hot encoded columns:

In [ ]:
for orig_col in cat_feat:
    cat_feat_ohe[orig_col] = [ohe_col for ohe_col in new_columns
                              if ohe_col.startswith(orig_col)]

In [ ]:
cat_feat_ohe

#### Save enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [ ]:
stream = open(f'{data_path}/cleaned/cat_feat_ohe_patient.yaml', 'w')
yaml.dump(cat_feat_ohe, stream, default_flow_style=False)

### Create mortality label

Combine info from discharge location and discharge status. Using the hospital discharge data, instead of the unit, as it has a longer perspective on the patient's status. I then save a feature called "deathOffset", which has a number if the patient is dead on hospital discharge or is NaN if the patient is still alive/unknown (presumed alive if unknown). Based on this, a label can be made later on, when all the tables are combined in a single dataframe, indicating if a patient dies in the following X time, according to how faraway we want to predict.

In [ ]:
patient_df.hospitaldischargestatus.value_counts()

In [ ]:
patient_df.hospitaldischargelocation.value_counts()

In [ ]:
tmp_col = patient_df.apply(lambda df: df['hospitaldischargeoffset']
                                      if df['hospitaldischargestatus'] == 'Expired'
                                      else np.nan, axis=1)
tmp_col

In [ ]:
tmp_col.index.value_counts()

In [ ]:
tmp_col[0][100783]

In [ ]:
(~tmp_col.isnull()).sum()

In [ ]:
len(patient_df)

In [ ]:
# Remove duplicate NaNs
tmp_col = tmp_col.loc[~tmp_col.index.duplicated(keep='first')]

In [ ]:
tmp_col.index.value_counts()

In [ ]:
tmp_col[0][100783]

In [ ]:
(~tmp_col.isnull()).sum()

In [ ]:
len(patient_df)

Something's wrong with this `apply` line, as it's creating duplicate and fake (in rows where there is a value) NaN rows. Moving on to a more complex solution, where we can filter out the fake NaNs and remove the duplicate NaNs.

In [ ]:
def get_death_ts(df):
    if df['hospitaldischargestatus'] == 'Expired':
        df['death_ts'] = df['hospitaldischargeoffset']
    else:
        df['death_ts'] = np.nan
    return df

In [ ]:
tmp_df = patient_df.copy()
tmp_df['death_ts'] = tmp_df['hospitaldischargeoffset']

In [ ]:
tmp_df = tmp_df.apply(get_death_ts, axis=1, result_type='broadcast')
tmp_df.head()

In [ ]:
tmp_df['death_ts'].index.value_counts()

In [ ]:
tmp_df['death_ts'][100783]

In [ ]:
(~tmp_df['death_ts'].isnull()).sum()

In [ ]:
tmp_col = tmp_df.groupby('patientunitstayid').death_ts.max()
tmp_col

In [ ]:
tmp_col.index.value_counts()

In [ ]:
(~tmp_col.isnull()).sum()

In [ ]:
patient_df['death_ts'] = tmp_col
patient_df.head()

Remove the now unneeded hospital discharge features:

In [ ]:
patient_df = patient_df.drop(['hospitaldischargeoffset', 'hospitaldischargestatus', 'hospitaldischargelocation'], axis=1)
patient_df.head(6)

### Create the timestamp feature and sort

Create the timestamp (`ts`) feature:

In [ ]:
patient_df['ts'] = 0
patient_df.head()

Sort by `ts` so as to be easier to merge with other dataframes later:

In [ ]:
# Index setting is failing on modin, so we're just going to skip this part for now
# patient_df = patient_df.set_index('ts')
# patient_df.head()

### Normalize data

Save the dataframe before normalizing:

In [ ]:
patient_df.to_csv(f'{data_path}cleaned/unnormalized/ohe/patient.csv')

In [ ]:
patient_df_norm, mean, std = du.data_processing.normalize_data(patient_df,
                                                               id_columns=['patientunitstayid', 'ts', 'death_ts'],
                                                               get_stats=True, inplace=True)
patient_df_norm.head(6)

Save a dictionary with the mean and standard deviation values of each column that was normalized:

In [ ]:
norm_stats = dict()
for key, _ in mean.items():
    norm_stats[key] = dict()
    norm_stats[key]['mean'] = mean[key]
    norm_stats[key]['std'] = std[key]
norm_stats

In [ ]:
stream = open(f'{data_path}/cleaned/patient_norm_stats.yaml', 'w')
yaml.dump(norm_stats, stream, default_flow_style=False)

In [ ]:
patient_df_norm.to_csv(f'{data_path}cleaned/normalized/ohe/patient.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
patient_df_norm.describe().transpose()

In [ ]:
# [TODO] Remove the rows with ts = 0 if there are no matching rows in other tables

## Notes data

### Initialize variables

In [ ]:
# List of categorical features
cat_feat = []
# Dictionary of the one hot encoded columns originary from each categorical feature, that will be embedded
cat_feat_ohe = dict()

### Read the data

In [ ]:
note_df = pd.read_csv(f'{data_path}original/note.csv')
note_df.head()

In [ ]:
len(note_df)

In [ ]:
note_df.patientunitstayid.nunique()

Get an overview of the dataframe through the `describe` method:

In [ ]:
note_df.describe().transpose()

In [ ]:
note_df.columns

In [ ]:
note_df.dtypes

### Check for missing values

In [ ]:
du.search_explore.dataframe_missing_values(note_df)

### Remove unneeded features

In [ ]:
note_df.notetype.value_counts().head(20)

In [ ]:
note_df.notepath.value_counts().head(40)

In [ ]:
note_df.notevalue.value_counts().head(20)

In [ ]:
note_df[note_df.notepath.str.contains('notes/Progress Notes/Social History')].head(20)

In [ ]:
note_df[note_df.notepath.str.contains('notes/Progress Notes/Social History')].notepath.value_counts().head(20)

In [ ]:
note_df[note_df.notepath.str.contains('notes/Progress Notes/Social History')].notevalue.value_counts().head(20)

Out of all the possible notes, only those addressing the patient's social history seem to be interesting and containing information not found in other tables. As such, we'll only keep the note paths that mention social history:

In [ ]:
note_df = note_df[note_df.notepath.str.contains('notes/Progress Notes/Social History')]
note_df.head()

In [ ]:
len(note_df)

There are still rows that seem to contain irrelevant data. Let's remove them by finding rows that contain specific words, like "obtain" and "print", that only appear in said irrelevant rows:

In [ ]:
category_types_to_remove = ['obtain', 'print', 'copies', 'options']

In [ ]:
du.search_explore.find_row_contains_word(note_df, feature='notepath', words=category_types_to_remove).value_counts()

In [ ]:
note_df = note_df[~du.search_explore.find_row_contains_word(note_df, feature='notepath', words=category_types_to_remove)]
note_df.head()

In [ ]:
len(note_df)

In [ ]:
note_df.patientunitstayid.nunique()

In [ ]:
note_df.notetype.value_counts().head(20)

Filtering just for interesting social history data greatly reduced the data volume of the notes table, now only present in around 20.5% of the unit stays. Still, it might be useful to include.

Besides the usual removal of row identifier, `noteid`, I'm also removing apparently irrelevant (`noteenteredoffset`, `notetype`) and redundant (`notetext`) columns:

In [ ]:
note_df = note_df.drop(['noteid', 'noteenteredoffset', 'notetype', 'notetext'], axis=1)
note_df.head()

### Separate high level notes

In [ ]:
note_df.notepath.value_counts().head(20)

In [ ]:
note_df.notepath.map(lambda x: x.split('/')).head().values

In [ ]:
note_df.notepath.map(lambda x: len(x.split('/'))).min()

In [ ]:
note_df.notepath.map(lambda x: len(x.split('/'))).max()

In [ ]:
note_df.notepath.apply(lambda x: du.search_explore.get_element_from_split(x, 1, separator='/')).value_counts()

In [ ]:
note_df.notepath.apply(lambda x: du.search_explore.get_element_from_split(x, 2, separator='/')).value_counts()

In [ ]:
note_df.notepath.apply(lambda x: du.search_explore.get_element_from_split(x, 3, separator='/')).value_counts()

In [ ]:
note_df.notepath.apply(lambda x: du.search_explore.get_element_from_split(x, 4, separator='/')).value_counts()

In [ ]:
note_df.notepath.apply(lambda x: du.search_explore.get_element_from_split(x, 5, separator='/')).value_counts()

In [ ]:
note_df.notepath.apply(lambda x: du.search_explore.get_element_from_split(x, 6, separator='/')).value_counts()

In [ ]:
note_df.notepath.apply(lambda x: du.search_explore.get_element_from_split(x, 7, separator='/')).value_counts()

In [ ]:
note_df.notevalue.value_counts()

There are always 8 levels of the notes. As the first 6 ones are essentially always the same ("notes/Progress Notes/Social History / Family History/Social History/Social History/"), it's best to just preserve the 7th one and isolate the 8th in a new feature. This way, the split provides further insight to the model on similar notes. However, it's also worth taking note that the 8th level of `notepath` seems to be identical to the feature `notevalue`. We'll look more into it later.

In [ ]:
note_df['notetopic'] = note_df.notepath.apply(lambda x: du.search_explore.get_element_from_split(x, 6, separator='/'))
note_df['notedetails'] = note_df.notepath.apply(lambda x: du.search_explore.get_element_from_split(x, 7, separator='/'))
note_df.head()

Remove the now redundant `notepath` column:

In [ ]:
note_df = note_df.drop('notepath', axis=1)
note_df.head()

Compare columns `notevalue` and `notedetails`:

In [ ]:
note_df[note_df.notevalue != note_df.notedetails]

The previous blank output confirms that the newly created `notedetails` feature is exactly equal to the already existing `notevalue` feature. So, we should remove one of them:

In [ ]:
note_df = note_df.drop('notedetails', axis=1)
note_df.head()

In [ ]:
note_df[note_df.notetopic == 'Smoking Status'].notevalue.value_counts()

In [ ]:
note_df[note_df.notetopic == 'Ethanol Use'].notevalue.value_counts()

In [ ]:
note_df[note_df.notetopic == 'CAD'].notevalue.value_counts()

In [ ]:
note_df[note_df.notetopic == 'Cancer'].notevalue.value_counts()

In [ ]:
note_df[note_df.notetopic == 'Recent Travel'].notevalue.value_counts()

In [ ]:
note_df[note_df.notetopic == 'Bleeding Disorders'].notevalue.value_counts()

Considering how only the categories of "Smoking Status" and "Ethanol Use" in `notetopic` have more than one possible `notevalue` category, with the remaining being only 2 useful ones (categories "Recent Travel" and "Bleeding Disorders" have too little samples), it's probably best to just turn them into features, instead of packing in the same embedded feature.

### Convert categories to features

Make the `notetopic` and `notevalue` columns of type categorical:

In [ ]:
# Only needed while using Dask, not with Modin or Pandas
# note_df = note_df.categorize(columns=['notetopic', 'notevalue'])

Transform the `notetopic` categories and `notevalue` values into separate features:

In [ ]:
note_df = du.data_processing.category_to_feature(note_df, categories_feature='notetopic',
                                                 values_feature='notevalue', min_len=1000, inplace=True)
note_df.head()

Now we have the categories separated into their own features, as desired. Notice also how categories `Bleeding Disorders` and `Recent Travel` weren't added, as they appeared in less than the specified minimum of 1000 rows.

Remove the old `notevalue` and `notetopic` columns:

In [ ]:
note_df = note_df.drop(['notevalue', 'notetopic'], axis=1)
note_df.head()

While `Ethanol Use` and `Smoking Status` have several unique values, `CAD` and `Cancer` only have 1, indicating when that characteristic is present. As such,we should turn `CAD` and `Cancer` into binary features:

In [ ]:
note_df['CAD'] = note_df['CAD'].apply(lambda x: 1 if x == 'CAD' else 0)
note_df['Cancer'] = note_df['Cancer'].apply(lambda x: 1 if x == 'Cancer' else 0)
note_df.head()

In [ ]:
note_df['CAD'].value_counts()

In [ ]:
note_df['Cancer'].value_counts()

In [ ]:
note_df['Smoking Status'].value_counts()

In [ ]:
note_df['Ethanol Use'].value_counts()

### Discretize categorical features

Convert binary categorical features into one hot encode columns, which can later be embedded or used as is.

#### One hot encode features

Update list of categorical features:

In [ ]:
cat_feat = ['Smoking Status', 'Ethanol Use']

In [ ]:
note_df[cat_feat].head()

Convert dataframe to Pandas, as the `one_hot_encoding_dataframe` isn't working properly with Modin:

In [ ]:
note_df, pd = du.utils.convert_dataframe(note_df, to='pandas', dtypes=dict(note_df.dtypes))

Apply one hot encoding:

In [ ]:
note_df, new_columns = du.data_processing.one_hot_encoding_dataframe(note_df, columns=cat_feat,
                                                                     join_rows=False,
                                                                     get_new_column_names=True,
                                                                     inplace=True)
note_df

In [ ]:
note_df.dtypes

Save the association between the original categorical features and the new one hot encoded columns:

In [ ]:
for orig_col in cat_feat:
    cat_feat_ohe[orig_col] = [ohe_col for ohe_col in new_columns
                              if ohe_col.startswith(orig_col)]

In [ ]:
cat_feat_ohe

#### Save enumeration encoding mapping

Save the dictionary that maps from the original categories/strings to the new numerical encondings.

In [ ]:
stream = open(f'{data_path}/cleaned/cat_feat_ohe_note.yaml', 'w')
yaml.dump(cat_feat_ohe, stream, default_flow_style=False)

### Create the timestamp feature and sort

Create the timestamp (`ts`) feature:

In [ ]:
note_df = note_df.rename(columns={'noteoffset': 'ts'})
note_df.head()

Remove duplicate rows:

In [ ]:
len(note_df)

In [ ]:
note_df = note_df.drop_duplicates()
note_df.head()

In [ ]:
len(note_df)

Sort by `ts` so as to be easier to merge with other dataframes later:

In [ ]:
note_df = note_df.sort_values('ts')
note_df.head()

Check for possible multiple rows with the same unit stay ID and timestamp:

In [ ]:
note_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='CAD', n=5).head()

In [ ]:
note_df[note_df.patientunitstayid == 3091883].head(10)

In [ ]:
note_df[note_df.patientunitstayid == 3052175].head(10)

We can see that there are up to 5 categories per set of `patientunitstayid` and `ts`. As such, we must join them. However, this is a different scenario than in the other cases. Since we created the features from one categorical column, it doesn't have repeated values, only different rows to indicate each of the new features' values. As such, we just need to sum the features.

### Join rows that have the same IDs

In [ ]:
note_df = du.embedding.join_repeated_rows(note_df, inplace=True)
note_df.head()

In [ ]:
note_df.dtypes

In [ ]:
note_df.groupby(['patientunitstayid', 'ts']).count().nlargest(columns='CAD', n=5).head()

In [ ]:
note_df[note_df.patientunitstayid == 3091883].head(10)

In [ ]:
note_df[note_df.patientunitstayid == 3052175].head(10)

Comparing the output from the two previous cells with what we had before the `join_repeated_rows` method, we can see that all rows with duplicate IDs have been successfully joined.

### Clean column names

Standardize all column names to be on lower case, have spaces replaced by underscores and remove comas.

In [ ]:
note_df.columns = du.data_processing.clean_naming(note_df.columns)
note_df.head()

### Save the dataframe

Save the dataframe before normalizing:

In [ ]:
# note_df.to_csv(f'{data_path}cleaned/unnormalized/ohe/note.csv')

Save the dataframe after normalizing:

In [ ]:
note_df.to_csv(f'{data_path}cleaned/normalized/ohe/note.csv')

Confirm that everything is ok through the `describe` method:

In [ ]:
note_df.describe().transpose()